In [2]:
from google import genai
from google.genai import types
import json
import os
from pathlib import Path
import time

from pydantic import BaseModel
from typing import List

# --- 1. 配置 Gemini ---
# 从环境变量中读取API密钥并配置
try:
    client = genai.Client()
except KeyError:
    print("错误：请先设置 GOOGLE_API_KEY 环境变量。")
    exit()

# --- 辅助函数 (与之前基本相同) ---

def load_data(json_path, accurate_text_path):
    """加载JSON时间戳数据和精确的文本文件"""
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            timing_data = json.load(f)
    except FileNotFoundError:
        print(f"错误: JSON文件未找到 at {json_path}")
        return None, None
    except json.JSONDecodeError:
        print(f"错误: JSON文件格式不正确 at {json_path}")
        return None, None
    try:
        with open(accurate_text_path, 'r', encoding='utf-8') as f:
            accurate_text = f.read()
    except FileNotFoundError:
        print(f"错误: 精确文本文件未找到 at {accurate_text_path}")
        return None, None
    return timing_data, accurate_text

def format_timing_data_for_prompt(segments_batch):
    lines = []
    for seg in segments_batch:
        lines.append(f'  - 时间: {seg["start"]:.3f} - {seg["end"]:.3f}, 文本: "{seg["text"]}"')
    return "\n".join(lines)

class SubtitleSegment(BaseModel):
    start: float
    end: float
    original_text: str
    translated_text: str

In [ ]:


# ------------ 配置 ------------
BATCH_SIZE            = 50
MODEL_FOR_CHUNKING    = "gemini-2.5-flash-lite"   # 用于提取片段的小模型
MODEL_FOR_ALIGNMENT   = "gemini-2.5-pro"        # 用于对齐+翻译的大模型

# 文本定位（切分）——提示词完全保留
CHUNK_SYSTEM_PROMPT = (
    """
        你是一个精通中文和日文的双语内容定位专家。
        你的唯一任务是：根据一段“参考中文文本”的含义，在一部“完整日文文本”中，找到并提取出与之语义完全对应的原始日文片段。

        # 你的工作流程：
        1.  **理解中文**：仔细阅读并完全理解“参考中文文本”所表达的核心意思。
        2.  **定位日文**：在“完整日文文本”中地毯式搜索，找到与中文含义完全匹配的日文句子或段落。
        3.  **精确提取**：完整地提取出你找到的那部分日文原文。为了保证上下文的连贯性，你可以适度地包含前后相邻的几个词或语句，切记只能多不能遗漏。
        4.  **纯净输出**：你的输出**必须**是纯粹的、未经修改的日文文本，除非原文有乱码非日文字符。**绝对不要**包含任何解释、标题、标签、引号或任何非日文内容。

        ---
        【一个完整的处理范例】
        [完整日文文本]:
        ...失礼します。ちょっとインタビューさせていただきたいなと思います。はい、夏目響です。デビューして何年目ぐらいですか？5周年になりましたので6年目です。いろんな役が演じてきて、セックス感も変わってきたんじゃないですか...

        [参考中文文本]:
        是的，我是夏目响。你出道几年了？因为已经5周年了，所以是第6年。

        [输出]:
        はい、夏目響です。デビューして何年目ぐらいですか？5周年になりましたので6年目です。
        ---

        现在，请严格遵循以上规则和范例，处理下面的实际任务。
    """)

chunk_config = types.GenerateContentConfig(
    system_instruction=CHUNK_SYSTEM_PROMPT,
    temperature=0.0,
)

# 对齐+翻译——提示词完全保留
ALIGN_SYSTEM_PROMPT = """
你是一个专业的字幕制作和翻译专家。你的任务是根据两份日文转录本，完成一个精确的、带时间戳的中文翻译。
一份是“带时间戳的不精确转录本”，它提供了准确的时间信息但文本内容可能有误。
另一份是“精确转录本”，它提供了准确的文本内容但没有时间信息。
你的工作流程如下：
1.  **对齐**：以“带时间戳的不精确转录本”为基础框架，将“精确转录本”中的文本内容，智能地填充到对应的时间段（segment）中。注意，两者的句子和词语不一定完全匹配，但是大部分是match的，尤其是一个segment里的第一个假名和最后一个假名，这是你匹配的重要参考，千万不能错配，你需要根据语义和上下文进行最佳的对齐，确保最终的文本流畅且符合逻辑，最重要的是，精细文本与粗文本的时间戳是一致的。
2.  **翻译**：将对齐好的、精确的日文文本内容，逐段翻译成流畅、自然的简体中文。
3.  **输出**：必须以一个JSON数组的格式返回结果，请把最终数组放入 JSON 对象的 `segments` 字段，仅返回该对象。不包含任何额外的解释。每个JSON对象包含以下字段：'start', 'end', 'original_text', 'translated_text'。
备注：
有时精确转录本中可能出现一些语气词在粗转录本中没有的情况，或者相反，这种情况下，忽略这些语气词，只对齐粗转录本中存在的内容。
"""

align_config = types.GenerateContentConfig(
    system_instruction=ALIGN_SYSTEM_PROMPT,
    temperature=0.1,
    response_schema=list[SubtitleSegment],
)



def extract_relevant_chunk_with_gemini(full_text, segment_batch):
    reference_text = " ".join([s["text"] for s in segment_batch])
    contents = (
        f"这是“完整文本”：\n---\n{full_text}\n---\n\n"
        f"请提取与下列“参考文本”对应的部分：\n---\n{reference_text}\n---"
    )
    try:
        rsp = client.models.generate_content(
            model=MODEL_FOR_CHUNKING,
            contents=contents,
            config=chunk_config
        )
        return rsp.text.strip()
    except Exception as e:
        print(f"Gemini 定位失败: {e}")
        return None

def align_and_translate_with_gemini(segment_batch, accurate_text_chunk):
    formatted_timings = segment_batch
    contents = f"""
--- 带时间戳的不精确转录本 (当前批次) ---
{formatted_timings}
---------------------------------

--- 精确转录本 (相关片段) ---
{accurate_text_chunk}
-------------------
"""
    try:
        rsp = client.models.generate_content(
            model=MODEL_FOR_ALIGNMENT,
            contents=contents,
            config=align_config
        )
        obj = json.loads(rsp.text)
        print(obj)
        if isinstance(obj, dict) and "segments" in obj:
            return obj["segments"]
        if isinstance(obj, list):
            return obj
        for v in obj.values():
            if isinstance(v, list):
                return v
    except Exception as e:
        print(f"Gemini 对齐/翻译失败: {e}")
        return None

def format_time_for_srt(sec):
    if sec is None or not isinstance(sec, (int, float)):
        sec = 0.0
    h, rem = divmod(sec, 3600)
    m, s = divmod(rem, 60)
    ms = int((s - int(s)) * 1000)
    return f"{int(h):02}:{int(m):02}:{int(s):02},{ms:03}"

def to_srt(all_segments):
    lines = []
    for i, item in enumerate(all_segments, 1):
        if not {"start","end","translated_text"} <= item.keys():
            print(f"警告: 第{i}条缺字段，已跳过")
            continue
        lines += [
            str(i),
            f"{format_time_for_srt(item['start'])} --> {format_time_for_srt(item['end'])}",
            item["translated_text"],
            ""
        ]
    return "\n".join(lines)

# ------------- 主流程 -------------
def main():
    json_file     = 
    accurate_file = 
    output_srt    = 

    timing_data, accurate_text = load_data(json_file, accurate_file)

    if not (timing_data and accurate_text):
        return

    segments = timing_data.get("segments", [])
    if not segments:
        print("segments 为空")
        return

    total_batches = (len(segments) + BATCH_SIZE - 1) // BATCH_SIZE
    all_results   = []

    for batch_idx in range(total_batches):
        batch = segments[batch_idx * BATCH_SIZE : (batch_idx + 1) * BATCH_SIZE]
        if not batch:
            continue
        print(f"\n=== 处理批次 {batch_idx + 1}/{total_batches} ===")

        # A. 提取精确文本片段

        chunk = extract_relevant_chunk_with_gemini(accurate_text, batch)
        if not chunk:
            print("  无法提取精确文本，跳过此批次")
            continue

        # B. 对齐 + 翻译
        timing_prompt = format_timing_data_for_prompt(batch)
        result = align_and_translate_with_gemini(timing_prompt, chunk)
        if result:
            all_results.extend(result)
            print(f"  成功获得 {len(result)} 条字幕")
        else:
            print("  对齐/翻译失败")

        time.sleep(1)  # 轻微延时减小速率限制风险

    if all_results:
        Path(output_srt).write_text(to_srt(all_results), encoding="utf-8")
        print(f"\n✅ SRT 已生成：{output_srt}")
    else:
        print("\n❌ 未获取到任何字幕")

if __name__ == "__main__":
    main()
